# Loading Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

# Loading the Data

In [2]:
defpred_train = pd.read_csv('Training Data.csv')
defpred_test = pd.read_csv('Test Data.csv')

# Cleaning The Data

In [3]:
defpred_train['married'] = defpred_train['married'] == 'single'
defpred_train["married"] = defpred_train['married'].astype(int)

defpred_test['married'] = defpred_test['married'] == 'single'
defpred_test["married"] = defpred_test['married'].astype(int)

In [ ]:
defpred_train['car_ownership'].unique()

In [4]:
defpred_train['house_ownership'] = defpred_train['house_ownership'].map({'rented': 0, 'norent_noown': 1, 
                                                                         'owned' : 2})

defpred_test['house_ownership'] = defpred_test['house_ownership'].map({'rented': 0, 'norent_noown': 1, 
                                                                         'owned' : 2})

In [5]:
defpred_train['car_ownership'] = defpred_train['car_ownership'].map({'no': 0, 'yes': 1, })
defpred_test['car_ownership'] = defpred_test['car_ownership'].map({'no': 0, 'yes': 1, })

# Feature Selection

In [6]:
train_data = defpred_train[['income','age','experience','married','house_ownership','car_ownership',
                            'current_job_years','current_house_years','risk_flag']]
#train_label = defpred_train[['risk_flag']]

test_data = defpred_test[['income','age','experience','married','house_ownership','car_ownership',
                            'current_job_years','current_house_years']]

In [7]:
X = train_data.iloc[:,:-1]
y = train_data.iloc[:,-1]

# Loading Some More Libraries

In [8]:
from sklearn.model_selection import KFold 
from collections import Counter
#from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler 
from sklearn.metrics import accuracy_score

In [ ]:
counter = Counter(y)
print(counter)

# Applying Data Augmentation For Imbalanced Data

In [ ]:
oversample = SVMSMOTE()

In [ ]:
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)

In [ ]:
X, y = oversample.fit_resample(X, y)

In [ ]:
oversample = ADASYN()

In [ ]:
X, y = under.fit_resample(X, y)

In [ ]:
counter = Counter(y)
print(counter)

# Normalizing the Data

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [ ]:
sc = StandardScaler()
sc.fit(X)
X_train_std = sc.transform(X)
X_test_std = sc.transform(test_data)

# Using Ensemble Model (RandomForestClassifier)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [17]:
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from numpy import mean

model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')

In [10]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, 
                            
                               max_features = 'sqrt',
                               random_state = 1)

In [ ]:
model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.001, max_depth= 40, random_state=None)

# Performing Cross Validation

In [ ]:
acc_score = []
k = 10
kf = KFold(n_splits=k, random_state=None)

In [ ]:
acc_score1 = []
k = 10
kf = KFold(n_splits=k, random_state=None)

In [11]:
over = SMOTE()

In [12]:
steps = [('over', over), ('model', model)]

In [13]:
pipeline = Pipeline(steps=steps)

In [18]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=None)

In [20]:
steps = [(‘over’, SMOTE()), (‘model’, RandomForestClassifier(n_estimators=100, criterion=’gini’, max_depth=None, random_state=1))]

SyntaxError: invalid character in identifier (<ipython-input-20-1f3071402cc2>, line 1)

In [ ]:
pipeline = Pipeline(steps=steps)
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)
acc = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

In [ ]:
scores = cross_val_score(pipeline, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('Mean ROC AUC: %.3f' % mean(scores))

# Training the Model

In [ ]:
c = 0
for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
    
#     if c == 2:  
    model1.fit(X_train,y_train)
    pred_values = model1.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    acc_score1.append(acc)
#         acc1 = acc
#     c += 1
     
avg_acc_score = sum(acc_score1)/k
#print('best accuracy: {}'.format(acc))
print('accuracy of each fold - {}'.format(acc_score1))
print('Avg accuracy : {}'.format(avg_acc_score))

In [ ]:
c = 0
for train_index , test_index in kf.split(X):
    X_train , X_test = X_train_std[train_index,:],X_train_std[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
    
#     if c == 2:  
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    print(acc)
    acc_score.append(acc)
#         acc1 = acc
#     c += 1
     
avg_acc_score = sum(acc_score)/k
#print('best accuracy: {}'.format(acc))
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

# Testing the Model

In [ ]:
pred = model.predict(X_test_std)

# Data Frame Creation and Saving the results of prediction in CSV File

In [ ]:
risk_flag = pd.DataFrame()

In [ ]:
risk_flag['risk_flag'] = pd.Series(pred)
risk_flag.insert(0, 'id',risk_flag.index + 1)

In [ ]:
risk_flag.to_csv('Prediction Dataset_final.csv', index=False)